# Лабораторная работа №2

**Требования:**
* Python >= 3.X

Лабораторную работу необходимо выполнять в данном шаблоне. Результатом работы будет являться файл (с измененным именем), который необходимо выложить в Moodle.

**Важно!!!** Имя файлу задавайте по следующему шаблону **lab_2_Группа_ФамилияИО.ipynb**. Например: если Вас зовут Иванов Иван Иванович, и Вы обучаетесь в группе 6207_010302D, то имя файла будет выглядеть так **lab_2_6407_010302D_ИвановИИ.ipynb**.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://disk.yandex.ru/i/DDb0zuyUmts5QA)). Откликами являются значения двух классов positive и negative.

In [1]:
# Код загрузки данных
# Если хотите добавить какие-либо библиотеки
# добавляйте их ИМЕННО ЗДЕСЬ
import pandas as pd
import nltk
import re
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)

    
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
imdb_data = pd.read_csv(r'input/IMDB Dataset.csv')
imdb_data.head(5)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\finnp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\finnp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\finnp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### Шаг №1 Подготовка данных

Обязательно предобработайте данные!



In [2]:
# вставьте код здесь
WNL = WordNetLemmatizer()
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление тегов
    text = text.replace('<br /><br />', '')
    # Удаление спецсимволов
    text = re.sub(r'[^a-z\s]', ' ', text)  # Удаляет всё кроме букв и пробелов
    # Токенизация
    words = word_tokenize(text)
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    # Лемматизация
    lemmatized_words = [WNL.lemmatize(word) for word in filtered_words]
    # Возвращаем обработанный текст
    return ' '.join(lemmatized_words)

# Применение функции предобработки к столбцу review
imdb_data['review'] = imdb_data['review'].apply(preprocess_text)


In [3]:
#TruncatedSVD
svd = TruncatedSVD(n_components=200, random_state=1)

В качестве исследуемых способов представления текстов необходимо рассмотреть:

#### 1.Компоненты вектора: частоты ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

In [4]:
# вставьте код здесь
# Векторизация текста с помощью CountVectorizer
count_vectorizer = CountVectorizer()
count_X = count_vectorizer.fit_transform(imdb_data['review'])


count_X = svd.fit_transform(count_X)
print(f"CountVectorizer with svd:{count_X.shape}")

CountVectorizer with svd:(50000, 200)


#### 2. Компоненты вектора: оценки tf-idf для слова ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). 

In [5]:
# вставьте код здесь
tfidf_vectorizer = TfidfVectorizer()
# Применение векторизатора к обработанным текстам
tfidf_X = tfidf_vectorizer.fit_transform(imdb_data['review'])
tfidf_X = svd.fit_transform(tfidf_X)
print(f"Tf-idf with svd:{tfidf_X.shape}")

Tf-idf with svd:(50000, 200)


#### 3. Компоненты вектора: частоты N-грам.

In [6]:
# вставьте код здесь
CountVectorizer_n = CountVectorizer(ngram_range=(2, 2))
count_n_X = CountVectorizer_n.fit_transform(imdb_data['review'])

count_n_X = svd.fit_transform(count_n_X)
print(f"CountVectorizer (n_gram) with svd :{count_n_X.shape}")

CountVectorizer (n_gram) with svd :(50000, 200)


### Шаг 2. Исследование моделей

<table>
		<tr>
			<td></td>
			<td>$y = 1$</td>
			<td>$y = 0$</td>
		</tr>
		<tr>
			<td>$a(x) = 1$</td>
			<td>True Positive (TP)</td>
			<td>False Positive (FP)</td>
		</tr>
    	<tr>
			<td>$a(x) = 0$</td>
			<td>False Negative (FN)</td>
			<td>True Negative (TN)</td>
		</tr>
</table>

В зависимости от способа представления оценить качество классификации как долю правильных ответов на выборке ($\operatorname{accuracy} = \frac{\operatorname{TP} + \operatorname{TN}}{\operatorname{TP} + \operatorname{TN} + \operatorname{FP} + \operatorname{FN}}$). Используйте перекрестную проверку ([cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)).

Для каждого из нижеперечисленных моделей необходимо определить оптимальные гиперпараметры ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

Качество классификации оцениваем для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

In [7]:
imdb_data['sentiment'] = imdb_data['sentiment'].replace({'positive': 1, 'negative': 0})


In [8]:
X_train_count, X_test_count, y_train, y_test = train_test_split(count_X, imdb_data['sentiment'], test_size=0.2, random_state=1)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(tfidf_X, imdb_data['sentiment'], test_size=0.2, random_state=1)
X_train_count_n, X_test_count_n, _, _ = train_test_split(count_n_X, imdb_data['sentiment'], test_size=0.2, random_state=1)

In [9]:
# вставьте код здесь
svc = SVC(max_iter = 7000, random_state=1)
parameters = {
    'C': [0.1, 1, 10],  # Примерные значения для параметра регуляризации
    'kernel': ['linear', 'rbf'],  # Ядра
    'gamma': ['scale', 'auto']  # Параметр ядра
}

grid_search_svc = GridSearchCV(svc, parameters, cv=3, scoring='accuracy')

# Для CountVectorizer
grid_search_svc.fit(X_train_count, y_train)
print(f"Лучшие параметры для CountVectorizer:, {grid_search_svc.best_params_}, Точность: {grid_search_svc.best_score_}")


# Для Tf-idf
grid_search_svc.fit(X_train_tfidf, y_train)
print(f"Лучшие параметры для Tf-idf:, {grid_search_svc.best_params_}, Точность: {grid_search_svc.best_score_}")


# Для CountVectorizer с n-gram
grid_search_svc.fit(X_train_count_n, y_train)
print(f"Лучшие параметры для CountVectorizer (n-gram):, {grid_search_svc.best_params_}, Точность: {grid_search_svc.best_score_}")


Лучшие параметры для CountVectorizer:, {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}, Точность: 0.8455999873073172
Лучшие параметры для Tf-idf:, {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}, Точность: 0.8756249860948788
Лучшие параметры для CountVectorizer (n-gram):, {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}, Точность: 0.7547250290676795


#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

In [11]:
# вставьте код здесь
rfc = RandomForestClassifier(n_estimators = 50, random_state=1)

parameters = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4] 
}
grid_search_rfc = GridSearchCV(rfc, parameters, cv=3, scoring='accuracy')
# Для CountVectorizer
grid_search_rfc.fit(X_train_count, y_train)
print(f"Лучшие параметры для Random Forest с CountVectorizer:, {grid_search_rfc.best_params_}, Точность: {grid_search_rfc.best_score_}")

# Для Tf-idf
grid_search_rfc.fit(X_train_tfidf, y_train)
print(f"Лучшие параметры для Random Forest с Tf-idf:, {grid_search_rfc.best_params_}, Точность: {grid_search_rfc.best_score_}")

# Для CountVectorizer с n-граммами
grid_search_rfc.fit(X_train_count_n, y_train)
print(f"Лучшие параметры для Random Forest с CountVectorizer (n-gram):, {grid_search_rfc.best_params_}, Точность: {grid_search_rfc.best_score_}")

Лучшие параметры для Random Forest с CountVectorizer:, {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}, Точность: 0.7555748853223355
Лучшие параметры для Random Forest с Tf-idf:, {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2}, Точность: 0.8070749578848965
Лучшие параметры для Random Forest с CountVectorizer (n-gram):, {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}, Точность: 0.7430500334279769


### Шаг 3. Сравнение результатов

Сравнить точность обученных моделей. Найти наиболее точную.

In [13]:
# вставьте код здесь
svc = SVC(C=1, gamma='auto', kernel='rbf', random_state=1)
svc.fit(X_train_count, y_train)
y_pred_svc = svc.predict(X_test_count)
print("SVC Accuracy (Count Vectorizer):", accuracy_score(y_pred_svc, y_test))

svc = SVC(C=1, gamma='scale', kernel='rbf', random_state=1)
svc.fit(X_train_tfidf, y_train)
y_pred_svc = svc.predict(X_test_tfidf)
print("SVC Accuracy (tfidf):", accuracy_score(y_pred_svc, y_test))

svc = SVC(C=1, gamma='scale', kernel='rbf', random_state=1)
svc.fit(X_train_count_n, y_train)
y_pred_svc = svc.predict(X_test_count_n)
print("SVC Accuracy (Count Vectorizer N-grams):", accuracy_score(y_pred_svc, y_test))

rfc = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators = 50, random_state=1)
rfc.fit(X_train_count, y_train)
y_pred_rfc = rfc.predict(X_test_count)
print("RFC Accuracy (Count Vectorizer):", accuracy_score(y_pred_rfc, y_test))

rfc = RandomForestClassifier(max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators = 50, random_state=1)
rfc.fit(X_train_tfidf, y_train)
y_pred_rfc = rfc.predict(X_test_tfidf)
print("RFC Accuracy (tfidf):", accuracy_score(y_pred_rfc, y_test))

rfc = RandomForestClassifier(max_depth=None, min_samples_leaf=4, min_samples_split=2, n_estimators = 50, random_state=1)
rfc.fit(X_train_count_n, y_train)
y_pred_rfc = rfc.predict(X_test_count_n)
print("RFC Accuracy (Count Vectorizer N-grams):", accuracy_score(y_pred_rfc, y_test))

SVC Accuracy (Count Vectorizer): 0.8487
SVC Accuracy (tfidf): 0.8758
SVC Accuracy (Count Vectorizer N-grams): 0.7863
RFC Accuracy (Count Vectorizer): 0.7551
RFC Accuracy (tfidf): 0.8088
RFC Accuracy (Count Vectorizer N-grams): 0.7545
